In [1]:
import sys
sys.path.insert(0, "../../../")

In [2]:
import deeplay as dtm

import torch
import torch.nn as nn
import torchvision
import pytorch_lightning as pl

In [3]:
# Load the MNIST dataset
mnist = torchvision.datasets.MNIST(
    root="data", train=True, download=True, transform=torchvision.transforms.ToTensor()
)

mnist_test = torchvision.datasets.MNIST(
    root="data", train=False, download=True, transform=torchvision.transforms.ToTensor()
)

In [4]:
mnist_dataloader = torch.utils.data.DataLoader(mnist, batch_size=128, num_workers=2)
mnist_test_dataloader = torch.utils.data.DataLoader(mnist_test, batch_size=128, num_workers=4)

#### 1.1 Default ImageClassifier

In [5]:
classifier = dtm.ImageClassifier(num_classes=10)
print(classifier)
# Not necessary, but to get the number of parameters we first need to run the model with a batch of data
# This is because the model is initialized lazily
classifier(torch.rand(1, 1, 28, 28))

trainer = pl.Trainer(max_epochs=500000, accelerator="cuda")
trainer.fit(classifier, mnist_dataloader)
trainer.test(dataloaders=mnist_test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


ImageClassifier(
  (backbone): ImageToImageEncoder(
    (blocks): ModuleList(
      (0): Template(
        (layer): LazyConv2d(0, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Template(
        (layer): LazyConv2d(0, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Template(
        (layer): LazyConv2d(0, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (3): Template(
        (layer): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): ReLU()
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mod

c:\Users\GU\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                          | Params
---------------------------------------------------------------
0 | backbone     | ImageToImageEncoder          | 24.4 K
1 | connector    | Flatten                       | 0     
2 | head         | CategoricalClassificationHead | 650   
3 | val_accuracy | Accuracy                      | 0     
---------------------------------------------------------------
25.0 K    Trainable params
0         Non-trainable params
25.0 K    Total params
0.100     Total estimated model params size (MB)
c:\Users\GU\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloa

Training: 0it [00:00, ?it/s]

# 1.2 With LeakyReLU

In [12]:
classifier = dtm.ImageClassifier(
    num_classes=10,
    backbone=dtm.Config()
                .__.activation(nn.LeakyReLU, negative_slope=0.1)
)
print(classifier)

# Not necessary, but to get the number of parameters we first need to run the model with a batch of data
# This is because the model is initialized lazily
classifier(torch.rand(1, 1, 28, 28))

trainer = pl.Trainer(max_epochs=5, accelerator="cuda")
trainer.fit(classifier, mnist_dataloader, mnist_test_dataloader)
trainer.test(dataloaders=mnist_test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                          | Params
---------------------------------------------------------------
0 | backbone     | ImageToImageEncoder          | 24.4 K
1 | connector    | Flatten                       | 0     
2 | head         | CategoricalClassificationHead | 650   
3 | val_accuracy | Accuracy                      | 0     
---------------------------------------------------------------
25.0 K    Trainable params
0         Non-trainable params
25.0 K    Total params
0.100     Total estimated model params size (MB)


ImageClassifier(
  (backbone): ImageToImageEncoder(
    (blocks): ModuleList(
      (0): Template(
        (layer): LazyConv2d(0, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): LeakyReLU(negative_slope=0.1)
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Template(
        (layer): LazyConv2d(0, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): LeakyReLU(negative_slope=0.1)
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (2): Template(
        (layer): LazyConv2d(0, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): LeakyReLU(negative_slope=0.1)
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (3): Template(
        (layer): LazyConv2d(0, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activation): LeakyReLU(negative_s

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
Restoring states from the checkpoint path at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_25\checkpoints\epoch=4-step=9375.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_25\checkpoints\epoch=4-step=9375.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9818999767303467     │
│         test_loss         │    1.4790576696395874     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9818999767303467, 'test_loss': 1.4790576696395874}]

# 1.3 With BatchNorm2d

In [7]:
block = dtm.Layer("layer") >> dtm.Layer("activation") >> dtm.Layer("normalization") >> dtm.Layer("pool")

classifier = dtm.ImageClassifier(
    num_classes=10,
    backbone=dtm.Config()
                .blocks(block)
                .blocks.activation(nn.LeakyReLU, negative_slope=0.1)
                .blocks.normalization(nn.LazyBatchNorm2d)
                
)

# Not necessary, but to get the number of parameters we first need to run the model with a batch of data
# This is because the model is initialized lazily
classifier(torch.rand(1, 1, 28, 28))

trainer = pl.Trainer(max_epochs=5, accelerator="cuda")
trainer.fit(classifier, mnist_dataloader, mnist_test_dataloader)
trainer.test(dataloaders=mnist_test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                          | Params
---------------------------------------------------------------
0 | backbone     | ImageToImageEncoder          | 24.6 K
1 | connector    | Flatten                       | 0     
2 | head         | CategoricalClassificationHead | 650   
3 | val_accuracy | Accuracy                      | 0     
---------------------------------------------------------------
25.3 K    Trainable params
0         Non-trainable params
25.3 K    Total params
0.101     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
Restoring states from the checkpoint path at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_19\checkpoints\epoch=4-step=9375.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_19\checkpoints\epoch=4-step=9375.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.9854999780654907     │
│         test_loss         │    1.4761897325515747     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.9854999780654907, 'test_loss': 1.4761897325515747}]

# 1.4 With ResNet blocks

In [10]:
resnet_block = (
       dtm.Layer("layer") >> dtm.Layer("normalization") >> dtm.Layer("activation") 
    >> dtm.Layer("layer") >> dtm.Layer("normalization") >> dtm.Layer("activation") 
)

residual_connection = dtm.Layer("residual_layer")

resnet_block = (resnet_block + residual_connection)
resnet_block_with_pool = resnet_block >> dtm.Layer("pool")


classifier = dtm.ImageClassifier(
    num_classes=10,
    backbone=dtm.Config()
                .depth(4)
                .blocks(resnet_block_with_pool)
                .blocks[3](resnet_block) # Avoid pooling in the last block
                .blocks.normalization(nn.LazyBatchNorm2d)
                .blocks.residual_layer(nn.LazyConv2d, kernel_size=1, padding=0)
                .blocks[:4].populate("residual_layer.out_channels", lambda i: 16 * 2**i)
                .blocks[:4].populate("layer.out_channels", lambda i: 16 * 2**i)

                
)
print(classifier)

# Not necessary, but to get the number of parameters we first need to run the model with a batch of data
# This is because the model is initialized lazily
classifier(torch.rand(1, 1, 28, 28))

trainer = pl.Trainer(max_epochs=5, accelerator="cuda")
trainer.fit(classifier, mnist_dataloader, mnist_test_dataloader)
trainer.test(dataloaders=mnist_test_dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type                          | Params
---------------------------------------------------------------
0 | backbone     | ImageToImageEncoder          | 305 K 
1 | connector    | Flatten                       | 0     
2 | head         | CategoricalClassificationHead | 11.5 K
3 | val_accuracy | Accuracy                      | 0     
---------------------------------------------------------------
316 K     Trainable params
0         Non-trainable params
316 K     Total params
1.267     Total estimated model params size (MB)


ImageClassifier(
  (backbone): ImageToImageEncoder(
    (blocks): ModuleList(
      (0): Template(
        (LayerAdd): _TorchAdd(
          (a): Template(
            (layer): LazyConv2d(0, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (normalization): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activation): ReLU()
            (layer(0)): LazyConv2d(0, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (normalization(0)): LazyBatchNorm2d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (activation(0)): ReLU()
          )
          (b): LazyConv2d(0, 16, kernel_size=(1, 1), stride=(1, 1))
        )
        (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
      (1): Template(
        (LayerAdd): _TorchAdd(
          (a): Template(
            (layer): LazyConv2d(0, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
c:\Users\GU\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\connectors\checkpoint_connector.py:134: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_23\checkpoints\epoch=4-step=9375.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at c:\Users\GU\DeepTorch\examples\vision\classification\lightning_logs\version_23\checkpoints\epoch=4-step=9375.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │     0.984499990940094     │
│         test_loss         │    1.4766920804977417     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.984499990940094, 'test_loss': 1.4766920804977417}]